# Прогнозирование. Подготовка данных. 

> 🚀 В этой практике нам понадобятся: `etna==2.10.0, numpy==1.26.4, pandas==1.5.3, matplotlib==3.10.3, seaborn==0.13.2` 

> 🚀 Установить вы их можете с помощью команды: `%pip install etna==2.10.0 numpy==1.26.4 pandas==1.5.3 matplotlib==3.10.3 seaborn==0.13.2` 


## Содержание

* [Загрузка исходных данных](#Загрузка-исходных-данных)
* [Подготовка обучающей и тестовой выборок](#Подготовка-обучающей-и-тестовой-выборок)
  * [Обработка пропущенных значений](#Обработка-пропущенных-значений)
* [Обработка выбросов (аномалий)](#Обработка-выбросов-аномалий)
* [Заключение](#Заключение)
* [Вопросы для закрепления](#Вопросы-для-закрепления)


Привет! В предыдущем ноутбуке мы познакомились с данными и разобрались с формулировкой решаемой задачи. 

В этот раз мы поговорим о: 
* подготовке данных для прогнозирования;
* разделении данных на выборки.

Казалось бы, всё это знакомые шаги, которые вы уже видели и делали в предыдущем разделе с регрессией. Но как всегда, есть нюансы, на которые стоит обратить внимание. 

<p align="center"><img src="https://raw.githubusercontent.com/AleksDevEdu/ml_edu/master/assets/salt.jpg" width=600/></p>

 

In [ ]:
import warnings
warnings.filterwarnings("ignore")

from pathlib import Path 


from datetime import date, timedelta

from etna.datasets.tsdataset import TSDataset
from etna.transforms import MedianOutliersTransform, TimeSeriesImputerTransform

import numpy as np 
import pandas as pd 

import ipywidgets as widgets
from matplotlib import pyplot as plt
import seaborn as sns

sns.set_style("darkgrid")

## Загрузка исходных данных

Сначала загружаем наши исходные данные, которые использовали ранее для анализа. 
В этот раз сразу скажем `pandas` привести колонку `date` к типу `datetime` и отсортируем данные по времени от старых к новым.

In [ ]:
data_fpath = Path().cwd().parent / "datasets" / "ts_data.csv"

# parse_dates сразу преобразует указанные колонки в datetime 
df = pd.read_csv(data_fpath, index_col=0, parse_dates=["date"])

# сортировка по дате, по возрастанию
df = df.sort_values(by="date")

df.head(5)

И так как нам нужно подготовить выборки для обучения и тестирования, сразу сделаем папку, в которой всё это добро будет сохраняться. 

Папка должна появиться внутри папки `notebooks`, в которой вы сейчас и работаете. 

In [ ]:
output_dpath = Path().cwd() / "ts_datasets"
output_dpath.mkdir(parents=True, exist_ok=True)

## Подготовка обучающей и тестовой выборок

Разделим данные на 2 выборки: train и test. 

В случае прогнозирования нам не подойдёт способ, который был в "сырой" регрессии, потому что функция из `sklearn` `train_test_split` берёт рандомные строки из всей таблицы, а у нас данные "направлены" во времени. 

Поэтому принято выбирать конкретный день, по которому и будет деление. 
В нашем случае, предположим, что мы хотим, чтобы модель предсказывала нам количество пышек на продажу в течение 1 месяца (31 день). Поэтому и дату для деления на выборки возьмём 1 месяц с конца таблицы, т.е. 1 декабря 2016 года. 

> ⚠️ 1 декабря 2016 должно попасть в тестовую выборку

In [ ]:
SPLIT_DATE = date(2016, 12, 1)

In [ ]:
# TODO - сформируйте 2 выборки train_df и test_df, помните, что колонка date - это datetime

train_df = ... 
test_df = ... 

In [ ]:
assert np.allclose(train_df.shape, (227340, 6)), "Неверный размер обучающей выборки "
assert np.allclose(test_df.shape, (2790, 6)), "Неверный размер тестовой выборки"

print("Тесты прошли! Всё хорошо!")

### Обработка пропущенных значений

Выведем количество пропущенных целевых значений (`num_sold`) в каждой выборке

In [ ]:
# TODO - выведите кол-во пропущенных значений в num_sold в каждой выборке 

train_target_null = ... 
test_target_null = ...

train_target_null, test_target_null

In [ ]:
assert np.isclose(train_target_null, 8795), f"Неверное кол-во пропущенных значений в train выборке: {train_target_null} != 8795"
assert np.isclose(test_target_null, 76), f"Неверное кол-во пропущенных значений в test выборке: {test_target_null} != 76"

print("Тесты прошли! Всё хорошо!")

Пропущенные есть, как и ожидалось. Теперь повторим упражнение из анализа, когда группировали данные в сегменты.

In [ ]:
# TODO - добавьте в каждую выборку новую колонку segment в формате <country>_<store>_<product>
# Например, для пышечной Fluffy Bounce с шоколадными пышками в Канаде, должно получиться такое Canada_Fluffy Bounce_Chocolate 

train_df["segment"] = ...
test_df["segment"] = ...

In [ ]:
assert "segment" in train_df.columns, f"Не хватает колонки segment: {train_df.columns}"
assert train_df.iloc[0]["segment"] == "Canada_Fluffy Bounce_Chocolate", "Неверный формат записи в сегменте"

assert "segment" in test_df.columns, f"Не хватает колонки segment: {test_df.columns}"
assert test_df.iloc[0]["segment"] == "Kenya_Pyshka_Coffee", "Неверный формат записи в сегменте"

print("Тесты прошли! Всё хорошо!")

И найдём полностью пустые (пропущенные) сегменты.

In [ ]:
# группируем по сегментам и считаем среднее от продаж
train_grouped = train_df.groupby(["segment"], as_index=False)["num_sold"].mean()

# там, где среднее = None, это наши пациенты 
train_empty_segments = train_grouped[train_grouped["num_sold"].isna()]

# делим сегмент обратно на части - чисто ради красоты 
train_empty_segments["country"] = train_empty_segments["segment"].apply(lambda x: x.split("_")[0])
train_empty_segments["store"] = train_empty_segments["segment"].apply(lambda x: x.split("_")[1])
train_empty_segments["product"] = train_empty_segments["segment"].apply(lambda x: x.split("_")[2])

train_empty_segments

In [ ]:
# То же самое, но для тестовой выборки 
test_grouped = test_df.groupby(["segment"], as_index=False)["num_sold"].mean()

test_empty_segments = test_grouped[test_grouped["num_sold"].isna()]
test_empty_segments["country"] = test_empty_segments["segment"].apply(lambda x: x.split("_")[0])
test_empty_segments["store"] = test_empty_segments["segment"].apply(lambda x: x.split("_")[1])
test_empty_segments["product"] = test_empty_segments["segment"].apply(lambda x: x.split("_")[2])

test_empty_segments

Сегменты в обучающей и тестовой выборках совпали, значит мы на верном пути. 

Есть много разных способов, что можно сделать в таком случае. 

Например, можно посмотреть на такие же пышечные в других странах, подобрать поправочный коэффициент для Канады и Кении и заполнить ряды по аналогии с другими странами. В предыдущем ноутбуке мы видели, что форма рядов действительно похожа для определённых сегментов.

В нашем случае мы пойдём по более простому пути. Удалим такие ряды, будем считать, что шоколадные пышки просто не готовятся в этих конкретных пышечных. 
Так что и предсказывать нашей модели ничего в этом случае не надо. 

In [ ]:
# TODO - удалите данные от пустых сегменов в каждой из выборок

filtered_train_df = ... 
filtered_test_df = ... 

In [ ]:
assert "Canada_Fluffy Bounce_Chocolate" not in filtered_train_df["segment"].unique(), "'Canada_Fluffy Bounce_Chocolate' всё ещё в обучающих данных"
assert "Kenya_Fluffy Bounce_Chocolate" not in filtered_train_df["segment"].unique(), "'Kenya_Fluffy Bounce_Chocolate' всё ещё в обучающих данных"

assert "Canada_Fluffy Bounce_Chocolate" not in filtered_test_df["segment"].unique(), "'Canada_Fluffy Bounce_Chocolate' всё ещё в тестовых данных"
assert "Kenya_Fluffy Bounce_Chocolate" not in filtered_test_df["segment"].unique(), "'Kenya_Fluffy Bounce_Chocolate' всё ещё в тестовых данных"

print("Тесты прошли! Всё хорошо!")

In [ ]:
filtered_train_df["num_sold"].isna().sum(), filtered_test_df["num_sold"].isna().sum()

Так, с этой проблемой разобрались. Но у нас всё ещё остались другие ряды, в которых пропущена часть целевых значений. 

Давайте заполним эти пропуски, но не совсем привычным способом 😉.

Для работы с временными рядами существует много специальных библиотек, в нашей практике будем использовать фреймворк [ETNA](https://docs.etna.ai/stable/index.html). 

В этой библиотеке уже есть множество функций для работы со временем, что очень упрощает жизнь. Но нужно привыкнуть к некоторым особенностям. 

Для начала давайте преобразуем нашу train `pandas` таблицу в ETNA-датасет.

> ⚠️ Тестовую выборку не трогаем, считаем, что это наше неприкосновенное сокровище 💎💎💎

Первое, что надо сделать - это переименовать колонки

* `date` в `timestamp`
* `num_sold` в `target`

Иначе ETNA будет ругаться, вот такой у неё интерфейс.

In [ ]:
# TODO - переименуйте колонки date -> timestamp и num_sold -> target в обучающей выборке 


In [ ]:
assert "timestamp" in filtered_train_df.columns, f"Нет колонки `timestamp` в обучающей выборке: {filtered_train_df.columns}"
assert "target" in filtered_train_df.columns, f"Нет колонки `target` в обучающей выборке: {filtered_train_df.columns}"

print("Тесты прошли! Всё хорошо!")

In [ ]:
ts_train_df = TSDataset(filtered_train_df, freq="1D")

А теперь заполним пропуски в рамках каждого сегмента при помощи скользящего среднего значения за 30 дней.

ETNA сама поймёт границы сегмента, используя колонку `segment`, которую мы сделали ранее. Название колонки здесь тоже важно, без `segment` ETNA будет ругаться, как матрос.

In [ ]:
imputer = TimeSeriesImputerTransform(in_column="target", strategy="running_mean", window=30)
ts_train_df.fit_transform([imputer])

Итак, какая-то магия случилась. Давайте на всякий случай проверим, что пропущенных значений действительно нет.

In [ ]:
# в датасете ETNA, привычная нам табличка сидит в свойстве df
ts_train_df.df.isna().sum().sum()

Ура, магия есть, а пропущенных значений - нет. Вот так просто, всего 2 строчки кода. 

## Обработка выбросов (аномалий)

Пропуски мы обработали и, пока у нас есть датасет в формате ETNA, давайте этим воспользуемся и заодно ещё и выбросы обработаем.

Для определения аномалий будем использовать медианный подход с окном в 9 дней. Найденные выбросы ETNA заменит на `None` 

> ⚠️ Будьте внимательны при использовании механизмов ETNA. Фреймворк не очень любит общаться и многое делает под капотом. Например, в нашем случае, попробуйте увеличить окно до 31 дня и проверить, что будет с данными после этого.

In [ ]:
outliers_remover = MedianOutliersTransform(in_column="target", window_size=9)
ts_train_df.fit_transform([outliers_remover])

print(f"Кол-во временных рядов с аномалиями: {len(outliers_remover.outliers_timestamps)}")

In [ ]:
null_vals = ts_train_df.df.isna().sum().sum()

print(f"Количество None в данных: {null_vals}")

Видим, что у нас получилось 85 рядов из 90 (на самом деле из 88, т.к. 2 ряда были полностью пустыми) с хотя бы одним аномальным значением. Всего 500 выбросов. 

Заменим эти выбросы на скользящее среднее значение с окном в 30 дней (как делали в первом шаге).

In [ ]:
# TODO - обработайте выбросы с None 


In [ ]:
null_vals = ts_train_df.df.isna().sum().sum()

assert np.isclose(null_vals, 0), f"Всё ещё есть пропущенные значения: {null_vals}"
print("Тесты прошли! Всё хорошо!")

print(f"Количество None в данных: {null_vals}")

Если вы видите 0 пропущенных значений - это победа, товарищи! 

Теперь преобразуем ETNA-датасет обратно в `pandas` формат. Ибо так удобнее жить.

In [ ]:
preprocessed_train_df = ts_train_df.to_pandas(flatten=True)

# Удалим вспомогательные колонки segment и id
preprocessed_train_df = preprocessed_train_df.drop(columns=["segment", "id"])

preprocessed_train_df.shape

In [ ]:
preprocessed_train_df.head()

Последняя проверка, которую нам нужно сделать - это убедиться, что после наших махинаций с пропусками и выбросами частота данных не изменилась (ресемплинг).

Ожидается, что разница во времени между каждой соседней строкой будет 1 день. Давайте проверим! 

In [ ]:
df_check = preprocessed_train_df.copy()
df_check["date_tt_shifted"] = df_check["timestamp"].shift()
df_check = df_check[~df_check["date_tt_shifted"].isna()]

ts_delta = (df_check["timestamp"] - df_check["date_tt_shifted"]).max()
print(f"Train дельта по времени: {ts_delta}")

assert ts_delta == timedelta(days=1), f"Дельта по времени больше 1 дня! Текущая дельта: {ts_delta}"

Ура! Успешный успех!

Последние косметические приготовления по тестовым данным и можно сохранять наши выборки.

Приведём тестовые данные к единому формату с обучающей выборкой. 

Для этого нам надо переименовать колонки и убрать лишнее.

In [ ]:
preprocessed_test_df = filtered_test_df.copy(deep=True)

In [ ]:
# TODO - удалите вспомогательные колонки segment и id 


In [ ]:
assert "segment" not in preprocessed_test_df.columns, "Колонка 'segment' всё ещё на месте"
assert "id" not in preprocessed_test_df.columns, "Колонка 'id' всё ещё на месте"

print("Тесты прошли! Всё хорошо!")

In [ ]:
# TODO - переименуйте date -> timestamp, num_sold -> target 


In [ ]:
assert "date" not in preprocessed_test_df.columns, "Колонка 'date' всё ещё на месте"
assert "timestamp" in preprocessed_test_df.columns, "Колонка 'timestamp' не найдена"

assert "num_sold" not in preprocessed_test_df.columns, "Колонка 'num_sold' всё ещё на месте"
assert "target" in preprocessed_test_df.columns, "Колонка 'target' не найдена"

print("Тесты прошли! Всё хорошо!")

In [ ]:
preprocessed_test_df.head()

И, наконец, сохраним полученные таблицы в csv-файлы.

In [ ]:
# Во имя красоты и перфекционизма! 
column_order = ["timestamp", "country", "store", "product", "target"]

train_fpath = output_dpath / "train.csv"
preprocessed_train_df[column_order].to_csv(train_fpath)

test_fpath = output_dpath / "test.csv"
preprocessed_test_df[column_order].to_csv(test_fpath)

Данные готовы и ждут нас в следующей части.

## Заключение

В этом ноутбуке вы узнали, что: 

* в прогнозировании разделение на выборки - это совсем другая история, нежели в стандартной регрессии. 
* ⚠️ Деление данных в прогнозировании **НЕ РАВНО** делению данных в регрессии. 
* для подготовки данных есть много полезных механизмов по работе со временем в фреймворке ETNA. Но нужно быть аккуратным, т.к. эта штука много делает втихаря. 

Мы посмотрели только на небольшую часть возможных подходов, в реальной жизни вам скорее всего понядобятся ещё какие-то ухищрения для работы с данными. Всё зависит от проблем, с которыми вы столкнётесь, но рассмотренные методологии могут быть отправной точкой. 

## Вопросы для закрепления

1. Можно ли при решении задачи прогнозирования разделить данные на выборки с использовать функции `train_test_split` из `sklearn`? 
2. Что будет если подать временной ряд, состоящий только из `None`, в `TimeSeriesImputerTransform`?
3. Зачем нужен `segment` в ETNA? 
4. Можно ли делать `fit_transform` на тестовой выборке? 